# Embed complaints and then use embeddings to predict positive or negative

In [5]:
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os


In [6]:
# get the API key from the json file
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [28]:
import pandas as pd
import openai
import json

# read in the credit_risk_dataset.json file into a pandas dataframe
df = pd.read_json('credit_risk_dataset.json')

# loop over the dataframe and calcula   te the embedding for each row
embeddings = []
for index, row in df.iterrows():
    model="text-embedding-ada-002"
    vec = openai.Embedding.create(input = [row['prompt']], model=model)['data'][0]['embedding']
    embeddings.append(vec)
    #embeddings = np.append(embeddings, [vec], axis=0)
print (type(vec))


<class 'list'>


### Dataframe of embeddings causes problems. We need to be careful because "complaints.embeddings" is a Pandas Series and we need a Numpy matrix ie array of arrays. 

In [38]:
# plot the embeddings clusters using TSNE - remember complaints is a dataframe
from sklearn.manifold import TSNE
import matplotlib
from ast import literal_eval
import numpy as np
from ast import literal_eval
tsne = TSNE(n_components=2, random_state=3)

# transform embeddings from list of lists to numpy array of arrays
for i in range(len(embeddings)):
    embeddings[i] = np.array(embeddings[i])
matrix = np.array(embeddings)
print (type(matrix))
print(matrix[0][0:10])
X_2d = tsne.fit_transform(matrix)
print(X_2d.shape)
# use TSNE to reduce the dimensionality of the embeddings
#tsne_embeddings = tsne.fit_transform(matrix)


<class 'numpy.ndarray'>
[-0.02990316  0.00016384  0.01256184 -0.0466244  -0.03112938  0.0054344
 -0.01769665  0.01177454 -0.03519822 -0.03606215]


ValueError: perplexity must be less than n_samples

In [37]:


colors = ["red", "darkorange",  "blue", "darkgreen","purple"]
x = [x for x,y in X_2d]
y = [y for x,y in X_2d]
color_indices = embeddings.Area.values 
print(color_indices)
colormap = matplotlib.colors.ListedColormap(colors)
plt.scatter(x, y, c=color_indices, cmap=colormap, alpha=0.7)
plt.show()

ValueError: perplexity must be less than n_samples

In [179]:
# split the data into training and test sets for classification of Score from the embeddings
# Score = sentiment,  Area = product category
from sklearn.model_selection import train_test_split

matrix = np.array(complaints.embeddings.apply(literal_eval).to_list())
X_train, X_test, y_train, y_test = train_test_split(
    complaints.embeddings.apply(literal_eval).to_list(),
    complaints.Score.values,  # Score = +1 or -1 AND Area = 0,1,2,3 
    test_size = 0.2,
    random_state=42
)
# print lenght of training set
print(len(X_train))

80


In [180]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=10) # n_estimators is the number of trees
rfr.fit(X_train, y_train)
preds = rfr.predict(X_test)
print(preds)
# these should be +1 or -1 for the sentiment and Areas 0,1,2,3 for the product categories

[ 0.8 -0.8  1.   1.  -0.8  1.  -0.8 -1.   1.  -1.   0.   0.6 -1.  -1.
  0.4  1.   1.  -1.   1.   1. ]


In [181]:
# compare the predictions to the actual scores
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test, preds))
print(rmse)

0.28982753492378877
